<br>
    <center>МИНИСТЕРСТВО НАУКИ И ВЫСШЕГО ОБРАЗОВАНИЯ</center>
    <center>федеральное государственное автономное образовательное учреждение высшего образования </center> <center>«Самарский национальный исследовательский университет имени академика С.П. Королева»</center>
    <center>(Самарский университет)</center> </br>

<br/>

<br>
<center>Институт 	     информатики и кибернетики</center>                                                   	  
<center>Кафедра 	     технической кибернетики</center>                                                              	
</br>
<br/>

<br>

<br/>
<br/>
<br/>
<br>
<center>ОТЧЕТ</center>
<center>по лабораторной работе №2</center>

<center>«Обработка данных с использование Kafka»</center>
<center>(дополнительный код для первого обработчика схемы two consumers launched in a group)</center>
<br/>
<center>по дисциплине <strong>«Большие данные»</strong></center>
<br/>
<center></center>
</br>
<br/>
<br/>
<br/>



<br>
<p style="text-align:right;">Выполнил: Фамилия И.О.
<br>613х-010402D
<br>    
<br>Преподаватель: Попов С.Б.
</p>
<br/>
<br/>
<br/>
    <br/>
<br/>
<br/>
<center>Самара 2025</center>


### Экспорт необходимых модулей

In [1]:
from kafka import KafkaProducer
from kafka import KafkaConsumer
from kafka.errors import KafkaError
from kafka.structs import TopicPartition

from kafka.admin import KafkaAdminClient, NewTopic, NewPartitions


### Определение списка серверов kafka и имени топика с двумя разделами

In [2]:
bootstrap_servers = 'localhost:9092'
topic2 = "rides2-events"

## Реализация схемы с двумя обработчиками (two consumers launched in a group)

### Реализация обработчика данных (consumer)

1. Создаём экземпляр класса KafkaConsumer (не забываем в параметрах определить параметры *group_id*, *consumer_timeout_ms* и десериализаторы для key- и value-составляющих в сообщении). Параметр *consumer_timeout_ms* необходимо определить с учётом количества сообщений и времени задержки при отправке (двух секунд может не хватить, поскольку иногда много сообщений подряд посылаются в один раздел).
2. Создаём массивы для накопления значений trip_count, passenger_count, trip_distance, total_amount с нулевыми значениями и соответствующим типом данных
3. В цикле получения сообщений объекта KafkaConsumer:
   * разделяем получаемую value-строку на отдельные элементы,
   * обновляем значения в массивах-накопителях, используя (key - 1) как индекс соответствующего массива 
4. Распечатываем отдельно для каждого перевозчика:
   * количество поездок(trip_count),
   * число перевезённых пассажиров(passenger_count),
   * расстояние поездок(trip_distance)
   * сумму оплаты(total_amount)
5. Завершаем работу объекта KafkaProducer

In [3]:
consumer1 = KafkaConsumer(topic2,
                          group_id ='rides2',
                          auto_offset_reset='earliest',
                          consumer_timeout_ms=15000,           # здесь необходимо скорректировать значение таймаута
                          key_deserializer= lambda key: int(key.decode('utf-8')), # обратите внимание на преобразование типа ключа
                          value_deserializer= lambda value: value.decode('utf-8'),
                          bootstrap_servers=[bootstrap_servers])

In [4]:
trip_count = 0
passenger_count = 0
trip_distance = 0.0
total_amount = 0.0

for msg in consumer1:
    parts = msg.value.split(",")

    trip_count += 1
    passenger_count += int(parts[3])
    trip_distance += float(parts[4])
    total_amount += float(parts[16])

print(f"\nПеревозчик VendorID = {msg.key}:")
print(f"Количество поездок: {trip_count}")
print(f"Число пассажиров: {passenger_count}")
print(f"Дистанция поездок: {trip_distance:.2f} миль")
print(f"Общая сумма оплаты: {total_amount:.2f} $")


Перевозчик VendorID = 2:
Количество поездок: 179
Число пассажиров: 242
Дистанция поездок: 744.65 миль
Общая сумма оплаты: 3709.17 $


In [5]:
consumer1.close()